In [1]:
import numpy as np
import win32gui, win32ui, win32con
import numpy as np
import os
from time import time
import cv2 as cv
import pyautogui
class windowcap:

    # properties
    w = 0
    h = 0
    wndw = None
    cropped_x = 0
    cropped_y = 0
    offset_x = 0
    offset_y = 0

    # constructor
    def __init__(self, window_name=None,**kwargs):
        #default to desktop if no specified window found
        if window_name==None:
            self.wndw = win32gui.GetDesktopWindow()
        else:
            # find the handle for the window we want to capture
            self.wndw = win32gui.FindWindow(None, window_name)
            if not self.wndw:
                raise Exception('Window not found: {}'.format(window_name))

        # get the window size
        window_rect = win32gui.GetWindowRect(self.wndw)
        self.w = window_rect[2] - window_rect[0]
        self.h = window_rect[3] - window_rect[1]

        # remove window border
        border_pixels = 8
        titlebar_pixels = 30
        self.w = self.w - (border_pixels * 2)
        self.h = self.h - titlebar_pixels - border_pixels
        self.cropped_x = border_pixels
        self.cropped_y = titlebar_pixels

        # account for cropping
        self.offset_x = window_rect[0] + self.cropped_x
        self.offset_y = window_rect[1] + self.cropped_y

    def get_screen(self):

        # get the window image data
        wDC = win32gui.GetWindowDC(self.wndw)
        dcObj = win32ui.CreateDCFromHandle(wDC)
        cDC = dcObj.CreateCompatibleDC()
        dataBitMap = win32ui.CreateBitmap()
        dataBitMap.CreateCompatibleBitmap(dcObj, self.w, self.h)
        cDC.SelectObject(dataBitMap)
        cDC.BitBlt((0, 0), (self.w, self.h), dcObj, (self.cropped_x, self.cropped_y), win32con.SRCCOPY)

        # convert the raw data into a format opencv can read
        #dataBitMap.SaveBitmapFile(cDC, 'debug.bmp')
        signedIntsArray = dataBitMap.GetBitmapBits(True)
        img = np.fromstring(signedIntsArray, dtype='uint8')
        img.shape = (self.h, self.w, 4)
        img = img[...,:3]
        img = np.ascontiguousarray(img)
        
        # free resources
        dcObj.DeleteDC()
        cDC.DeleteDC()
        win32gui.ReleaseDC(self.wndw, wDC)
        win32gui.DeleteObject(dataBitMap.GetHandle())

        return img
    
    @staticmethod
    def get_windows():
        def winEnumHandler(wndw, ctx):
            if win32gui.IsWindowVisible(wndw):
                print(hex(wndw), win32gui.GetWindowText(wndw)+".")
        win32gui.EnumWindows(winEnumHandler, None)

    def get_screen_position(self, pos):
        return (pos[0] + self.offset_x, pos[1] + self.offset_y)

In [2]:
# import cv2 as cv

# from windowcapture import windowcap

# Change the working directory to the folder this script is in.
# os.chdir(os.path.dirname(os.path.abspath(r'C:\Users\notal\Documents\GitHub\Guilty-Gear-Segmentation\Media\ScreenCapTest')))


# initialize the WindowCapture class
wincap = windowcap(window_name='VALORANT  ')

loop_time = time()
while(True):

    # get an updated image of the game
    screenshot = wincap.get_screen()

    cv.imshow('GG-Val-Input', screenshot)

    # debug the loop rate
    print('FPS {}'.format(1 / (time() - loop_time)))
    loop_time = time()

    # press 'q' with the output window focused to exit.
    # waits 1 ms every loop to process key presses
    if cv.waitKey(1) == ord('q'):
        cv.destroyAllWindows()
        break
    #screenshot key to create input images for modeling
    elif cv.waitKey(1) == ord('p'):
        cv.imwrite(f'Media/MatchTemplate/{loop_time}.jpg',screenshot)


print('Done.')


<ipython-input-1-29e08b126eab>:61: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  img = np.fromstring(signedIntsArray, dtype='uint8')


FPS 28.56474273844792
FPS 47.60952575540875
FPS 32.25097653246394
FPS 32.2460175902577
FPS 34.552017859643634
FPS 41.65685738973254
FPS 25.63520459615561
FPS 30.296470724202194
FPS 66.65030986810743
FPS 71.41550458871805
FPS 30.28400205055632
FPS 31.255758497090014
FPS 23.803006656867698
FPS 43.21307219171449
FPS 25.635047917074125
FPS 31.232484195006442
FPS 32.2623878897897
FPS 31.231553943870676
FPS 31.254826859020692
FPS 29.401319248266823
FPS 76.92583082679187
FPS 47.60952575540875
FPS 23.248733440496647
FPS 32.23015921804881
FPS 33.34290461313427
FPS 58.81129588603158
FPS 33.325684501581144
FPS 32.25048056960955
FPS 31.239462845311067
FPS 31.242953340087002
FPS 31.38767782442434
FPS 32.340249666520165
FPS 33.337339246824676
FPS 34.473920405043316
FPS 32.246265501149374
FPS 30.29603305307561
FPS 29.857797773285117
FPS 31.24341879832546
FPS 28.563575568131515
FPS 24.992426544633332
FPS 31.35881332615587
FPS 58.81129588603158
FPS 66.65030986810743
FPS 58.81047126291732
FPS 71.3778292

FPS 30.302380522342233
FPS 23.781008323316627
FPS 32.25023259390258
FPS 39.98726296822416
FPS 26.880360942346638
FPS 62.48404493042934
FPS 31.243884270432943
FPS 31.79791516621811
FPS 33.333630034650476
FPS 31.669704543223673
FPS 47.616552193903615
FPS 24.603198066612702
FPS 33.33548453755732
FPS 31.243186067472642
FPS 32.25097653246394
FPS 31.905553019930018
FPS 30.297346104393302
FPS 29.689983719119418
FPS 30.311578125790437
FPS 30.296689564507624
FPS 25.439296436694466
FPS 32.25072854912997
FPS 33.325154934053714
FPS 31.24109164581099
FPS 30.29603305307561
FPS 29.405029480015983
FPS 23.800575394235842
FPS 31.24621183902737
FPS 62.485906680174004
FPS 33.39414012738853
FPS 32.224711504479174
FPS 32.253456575567895
FPS 31.24784134339589
FPS 31.242487895716945
FPS 33.32462538335638
FPS 27.771881848940918
FPS 39.716155179107446
FPS 32.24998462200898
FPS 32.25023259390258
FPS 31.45783051203396
FPS 30.298002672734498
FPS 31.242720616168462
FPS 29.402555888146594
FPS 24.53455002778509
FPS 3

FPS 31.255758497090014
FPS 30.294063689483075
FPS 33.182522290171754
FPS 32.25097653246394
FPS 30.296470724202194
FPS 30.29625188705821
FPS 66.65030986810743
FPS 41.491200823037126
FPS 34.47108714947895
FPS 31.590275057994155
FPS 24.994511617374513
FPS 62.44497379704621
FPS 71.46539444539104
FPS 62.485906680174004
FPS 33.32621408593948
FPS 30.296470724202194
FPS 31.2150512026673
FPS 45.49136659436009
FPS 31.24807414360854
FPS 30.296908407974573
FPS 25.637868433599433
FPS 66.64925076671275
FPS 66.65454661030417
FPS 66.64819169897667
FPS 31.242720616168462
FPS 32.23858203563358
FPS 47.599260074673445
FPS 23.81354787061903
FPS 43.46158788054629
FPS 26.310102999661268
FPS 62.43474895428631
FPS 32.26437329805074
FPS 66.64925076671275
FPS 32.33476467640597
FPS 31.24737575337669
FPS 30.29712725460311
FPS 29.398022050422995
FPS 25.441611063933035
FPS 62.52502906890075
FPS 62.487768540865886
FPS 31.572678138596572
FPS 58.81047126291732
FPS 33.325684501581144
FPS 55.542660398596304
FPS 33.325154

FPS 22.722147883699638
FPS 45.52197790271115
FPS 26.313899432228112
FPS 32.246513415853
FPS 32.25072854912997
FPS 31.26344663088849
FPS 31.541337664876895
FPS 29.393489610708155
FPS 23.705204141610526
FPS 32.90037259285406
FPS 31.23574051042978
FPS 32.257425437989326
FPS 31.242953340087002
FPS 31.243186067472642
FPS 29.39967055689903
FPS 23.833033121764675
FPS 27.909556700070535
FPS 35.6965080553877
FPS 62.485906680174004
FPS 33.157601821401464
FPS 32.25072854912997
FPS 33.32409584948834
FPS 31.24411701168776
FPS 31.21969809747819
FPS 48.139564778257274
FPS 24.38506305122585
FPS 32.24453020495395
FPS 62.50639325206402
FPS 66.65242817187897
FPS 32.45134585180543
FPS 30.291438269598817
FPS 71.43861561520643
FPS 47.60790456407986
FPS 31.242953340087002
FPS 25.002408259615155
FPS 62.33916946583039
FPS 66.82020073283415
FPS 55.54118940106201
FPS 71.41428863311313
FPS 58.81129588603158
FPS 71.24688296245965
FPS 31.273703361269348
FPS 29.405854103130366
FPS 32.25122451961154
FPS 71.3839031945

FPS 52.61426528513008
FPS 23.251568840499367
FPS 43.436382842111804
FPS 26.32149558516213
FPS 31.242720616168462
FPS 33.15419456323266
FPS 32.26213973093756
FPS 31.24341879832546
FPS 28.561824991487914
FPS 71.34140699414887
FPS 47.31414132298529
FPS 23.807870672577522
FPS 43.468344197904464
FPS 31.57077694311048
FPS 27.771146320953978
FPS 31.24341879832546
FPS 32.25072854912997
FPS 30.18462092044187
FPS 28.4359593220339
FPS 26.05659474805708
FPS 58.81047126291732
FPS 31.242720616168462
FPS 71.4130727189144
FPS 31.242953340087002
FPS 29.88864897991178
FPS 30.31793211125889
FPS 24.295650965036263
FPS 32.25072854912997
FPS 55.54339592658315
FPS 71.4094252247344
FPS 32.24750511278889
FPS 31.605033531760984
FPS 57.080115948340385
FPS 33.32171315532322
FPS 49.988725344139205
FPS 24.105888100232765
FPS 44.32928543496412
FPS 25.642884217991515
FPS 31.24481525625745
FPS 58.99244715115543
FPS 66.68845994848475
FPS 32.25072854912997
FPS 55.54339592658315
FPS 58.80964666292765
FPS 33.3246253833563

FPS 31.23899750493427
FPS 31.809973076485534
FPS 33.322772088441155
FPS 32.25097653246394
FPS 45.4711462364892
FPS 31.23923017338974
FPS 24.384637745194937
FPS 31.634353292555076
FPS 62.48683759665092
FPS 62.48404493042934
FPS 55.54413147405083
FPS 33.32621408593948
FPS 62.491492595130964
FPS 62.485906680174004
FPS 58.807173001696505
FPS 33.32489015660133
FPS 29.40523563145866
FPS 29.391429872814548
FPS 24.80691751735885
FPS 38.91578136742779
FPS 26.923497618527982
FPS 62.48497579143389
FPS 31.242720616168462
FPS 66.64501469770399
FPS 31.23806686576946
FPS 30.298221535171994
FPS 31.243651532645536
FPS 28.564548203437848
FPS 23.800035181097538
FPS 34.483273453750215
FPS 32.248496870723194
FPS 31.83339152081847
FPS 31.72647917580672
FPS 65.90465415920305
FPS 49.987533817202376
FPS 32.2447780929757
FPS 29.17208474175465
FPS 24.483422060334362
FPS 33.34741127083068
FPS 32.72479304668055
FPS 32.23734310990185
FPS 62.48497579143389
FPS 31.241324345461994
FPS 32.25048056960955
FPS 33.32489015

FPS 23.258273445124654
FPS 33.70136997308264
FPS 33.83130741992466
FPS 31.242487895716945
FPS 43.46519098841427
FPS 25.833994431988963
FPS 62.47008534278608
FPS 31.945162494192555
FPS 30.29603305307561
FPS 34.47505383768144
FPS 31.242953340087002
FPS 32.25072854912997
FPS 44.42977447750601
FPS 26.30498780174225
FPS 32.248248925520706
FPS 31.242720616168462
FPS 49.987533817202376
FPS 27.77280131371588
FPS 24.383220165565994
FPS 23.186400950827828
FPS 28.564937276107713
FPS 35.70167344784733
FPS 34.48043866067098
FPS 58.81376989413167
FPS 76.90749399490254
FPS 32.248496870723194
FPS 30.295814222254325
FPS 30.296470724202194
FPS 29.40234977427586
FPS 24.982899211969954
FPS 32.54198573966747
FPS 32.607002923067355
FPS 32.25023259390258
FPS 30.29165703720823
FPS 30.204183919634175
FPS 29.40296812455748
FPS 24.378543321960603
FPS 33.32674368712953
FPS 32.25494478452121
FPS 33.325154934053714
FPS 28.557157835968244
FPS 71.35232975520134
FPS 47.65550543669685
FPS 23.799224907369055
FPS 43.5062

FPS 34.906282509008896
FPS 62.48311409715912
FPS 33.30425047046586
FPS 30.295814222254325
FPS 33.31933080186206
FPS 41.65975367500993
FPS 27.023935776092575
FPS 31.23806686576946
FPS 31.23829952036226
FPS 32.2455217799099
FPS 30.30062923069143
FPS 32.246513415853
FPS 29.398228103622294
FPS 29.40935926741365
FPS 24.752311877769976
FPS 58.823668008358695
FPS 32.58876181004475
FPS 66.70755137095234
FPS 27.020453915878036
FPS 35.70623238867086
FPS 71.41428863311313
FPS 47.60952575540875
FPS 33.32489015660133
FPS 71.41185684611979
FPS 45.444541957852536
FPS 21.271771048347425
FPS 41.64238200194595
FPS 62.518505269120126
FPS 66.64607366447389
FPS 62.48683759665092
FPS 62.48683759665092
FPS 62.48497579143389
FPS 63.61849868798252
FPS 66.65136900316229
FPS 55.54339592658315
FPS 76.87366433899672
FPS 47.620877187006826
FPS 32.25023259390258
FPS 29.60970823067637
FPS 43.46789371139576
FPS 66.65348737425906
FPS 66.61431929356458
FPS 66.68952029637639
FPS 62.48218329162198
FPS 55.540453931513014
F

FPS 33.31403790249559
FPS 28.568050239071503
FPS 32.25444869961088
FPS 33.325684501581144
FPS 66.6556058800159
FPS 76.90044369476733
FPS 31.24341879832546
FPS 66.53717657883465
FPS 31.268107946921127
FPS 32.246265501149374
FPS 33.187773478608335
FPS 32.43678996496709
FPS 33.32489015660133
FPS 32.25196850393701
FPS 32.2455217799099
FPS 31.243186067472642
FPS 32.25072854912997
FPS 28.564353671077452
FPS 33.38722875837804
FPS 27.771514080077335
FPS 32.24998462200898
FPS 30.296470724202194
FPS 25.63536127715232
FPS 33.32594929165641
FPS 32.246513415853
FPS 32.13459696758426
FPS 28.6996065551336
FPS 29.913375887030632
FPS 23.80516819623935
FPS 39.98268876962527
FPS 30.337229487327857
FPS 26.405343641205466
FPS 33.3270084940367
FPS 32.25023259390258
FPS 30.294720115565188
FPS 30.295157748757656
FPS 23.246285242394514
FPS 66.67149896677793
FPS 67.37837751004017
FPS 33.3389291618975
FPS 31.858538289759522
FPS 30.29165703720823
FPS 30.297346104393302
FPS 31.243186067472642
FPS 31.24248789571694

# Steps:

1. Image Segmentation model
    - Create pixel map labels of sample images
    - Exploratory analysis with unsupervised clustering of images to see what algorithm is picking up
    - Optimize reduction of features and kernels/layers to create output image to be used in classification
2. Reinforcement Learning of Guilty Gear